In [1]:
import numpy as np
import pandas as pd
from ace_cream import ace_cream
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [3]:
F_ace = pd.read_csv("F_dropna.csv",parse_dates = ['Date'],index_col=0)

In [4]:
F_ace.head(3)

,High,high_lag_1,high_lag_2,high_lag_3,high_lag_4,high_lag_5,high_lag_6,high_lag_7,high_lag_8,high_lag_9,...,vol_lag_22,vol_lag_23,vol_lag_24,vol_lag_25,vol_lag_26,vol_lag_27,vol_lag_28,vol_lag_29,vol_lag_30,Close_1
Date,,,,,,,,,,,,,,,,,,,,,
2002-07-26,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,8723.37,8635.66,8681.90,...,295015367.0,341228701.0,416432368.0,276960842.0,248083566.0,212707042.0,247554505.0,287001167.0,227552955.0,8711.88
2002-07-29,8711.88,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,8723.37,8635.66,...,407442095.0,295015367.0,341228701.0,416432368.0,276960842.0,248083566.0,212707042.0,247554505.0,287001167.0,8680.03
2002-07-30,8762.14,8711.88,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,8723.37,...,335305071.0,407442095.0,295015367.0,341228701.0,416432368.0,276960842.0,248083566.0,212707042.0,247554505.0,8736.59


In [5]:
feature_number = 4
feature_range = range(1,feature_number + 1)
max_lag = 30

In [6]:
def split_df(feature_number,max_lag,data):
    # Store each feature's dataframe:
    df_all = []
    # Define the number of features
    feature_range = range(1,feature_number + 1)
    # Compute the start and end index of each feature with lags in the dataframe
    idxs = dict(zip(feature_range,[i * (max_lag + 1) for i in feature_range]))
    # Target is the usually the last column of the origianl DataFrame
    target_series = data.iloc[:,-1]
    for i in feature_range:
        if i == 1 :
            df_all.append( pd.concat([data.iloc[:,:idxs[i]],target_series],axis=1) ) 
        else:
            df_all.append( pd.concat([data.iloc[:,idxs[i-1]:idxs[i]],target_series],axis=1) )
    print(idxs)
    return df_all

In [7]:
feature_df_list = split_df(feature_number,max_lag,F_ace)

{1: 31, 2: 62, 3: 93, 4: 124}


In [8]:
feature_df_list[0]

,High,high_lag_1,high_lag_2,high_lag_3,high_lag_4,high_lag_5,high_lag_6,high_lag_7,high_lag_8,high_lag_9,...,high_lag_22,high_lag_23,high_lag_24,high_lag_25,high_lag_26,high_lag_27,high_lag_28,high_lag_29,high_lag_30,Close_1
Date,,,,,,,,,,,,,,,,,,,,,
2002-07-26,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,8723.37,8635.66,8681.90,...,9413.08,9369.95,9430.66,9573.89,9733.39,9721.75,9687.77,9498.92,9625.40,8711.88
2002-07-29,8711.88,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,8723.37,8635.66,...,9160.81,9413.08,9369.95,9430.66,9573.89,9733.39,9721.75,9687.77,9498.92,8680.03
2002-07-30,8762.14,8711.88,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,8723.37,...,9269.92,9160.81,9413.08,9369.95,9430.66,9573.89,9733.39,9721.75,9687.77,8736.59
2002-07-31,8736.73,8762.14,8711.88,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,8621.95,...,9362.82,9269.92,9160.81,9413.08,9369.95,9430.66,9573.89,9733.39,9721.75,8506.62
2002-08-01,8734.31,8736.73,8762.14,8711.88,8267.72,8309.39,8202.02,7894.41,8103.86,8406.10,...,9327.44,9362.82,9269.92,9160.81,9413.08,9369.95,9430.66,9573.89,9733.39,8313.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-06,33235.37,33135.61,33248.61,33272.34,33240.22,33213.62,32774.14,32254.44,32014.86,31968.42,...,34117.74,33341.58,33224.95,33919.59,34054.79,33697.18,33909.51,34106.01,34727.38,33180.14
2022-06-07,33207.45,33235.37,33135.61,33248.61,33272.34,33240.22,33213.62,32774.14,32254.44,32014.86,...,33854.17,34117.74,33341.58,33224.95,33919.59,34054.79,33697.18,33909.51,34106.01,32910.90
2022-06-08,33156.50,33207.45,33235.37,33135.61,33248.61,33272.34,33240.22,33213.62,32774.14,32254.44,...,33055.84,33854.17,34117.74,33341.58,33224.95,33919.59,34054.79,33697.18,33909.51,32272.79


In [11]:
X = feature_df_list[0].iloc[:,5]
Y = feature_df_list[0].iloc[:,-1]

In [102]:
tx, ty = ace_cream(X, Y)

In [103]:
tx.shape

(5005, 10)

In [104]:
sum_tx_col = np.sum(tx,axis=1)
ty = ty.reshape(-1,)

In [105]:
stats.pearsonr(sum_tx_col, ty)

(0.9044634678080118, 0.0)

### RDC

In [2]:
from rdc import rdc

In [13]:
x_rdc = np.array(X)
y_rdc = np.array(Y)

In [14]:
print(rdc(x_rdc,y_rdc))

0.9976667283829344
